# Estonians on the r/place Canvas
data science project by Johan Erik Pukk, Jaanus Raudsik & Tanel Marran

Steps needed to complete:
1. Read in the data from *tile_placements_sorted.csv*
2. Try to identify the users who built the estonian flag and heart.
3. Map the contributions of the users identified in the previous task on the rest of the canvas
4. Create a heatmap of the contributions of the identified users
5. Identify regions and objects where the identified users contributed to
6. Making an informative and attractive poster about our findings from data

**<font color='red'>PS!<font>** The *tile_placements_sorted.csv* file is too large to commit to github. You'll need to download it into your local repository on your own.<br>
Link to dataset: http://skeeto.s3.amazonaws.com/share/tile_placements_sorted.csv.xz

### Helpful info
Reddit post about the r/place datasets: https://www.reddit.com/r/redditdata/comments/6640ru/place_datasets_april_fools_2017/

Here is a list of all Estonian structures that we have found on the canvas, with appropriated coordinates.
It may be beneficial to also add timesstamps, so we know when these structures existed.
The coordinates may not be super percise, so keep that in mind. These coords are simply for finding the ge

**Estonian Flag**<br>
Coordiantes: (156, 70)
![title](Images/flag_156_70.png)

**Estonian Heart**<br>
Coordiantes: (187, 118)
![title](Images/heart_187_118.png)

## Step 1
Import some libraries and read in the dataset. Create a dictionary for colors.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
raw_data = pd.read_csv("tile_placements_sorted.csv")
raw_data["x"] = raw_data["x_coordinate"]
raw_data["y"] = raw_data["y_coordinate"]
del raw_data["x_coordinate"]
del raw_data["y_coordinate"]

In [149]:
raw_data.sort_values("ts",ascending=False)

,ts,user,color,x,y
8951761,1491238734000,WcPEECDEtqTri0VN5AJggUWOi/A=,5,605,160
15110795,1491238733000,uLIYXambhhJnq+XQsp3/v4hOSg8=,3,31,16
1788175,1491238733000,4tD092krB0kr/mIgPzbkdWRbbuA=,15,159,602
10357844,1491238733000,c34iVZhk/TWfZvgLERa92dv4nXI=,10,33,70
776850,1491238733000,0zJQ/whk/woFFZpfA53DCstaOV8=,5,176,159
...,...,...,...,...,...
4776829,1490919349000,GP2B74SRyrZ3HIflSFoAzM+jS3U=,11,477,495
4776825,1490919337000,GP2B74SRyrZ3HIflSFoAzM+jS3U=,11,474,495
1256304,1490918781000,2puPLvQ6X2zTSpfBkHXhxlIVo40=,0,518,498
1256296,1490918728000,2puPLvQ6X2zTSpfBkHXhxlIVo40=,1,490,504


Notice that the colors in this dataset have numeric identifiers and thus we don't really know what number corresbonds to what color.

In [15]:
print("Total number of colors:", len(raw_data.color.value_counts()))

Total number of colors: 16


An array of color strings should be created where the index of the color in the array corresbonds to its identifier in the dataset.
Thankfully these colors are documented on the reddit post linked earlier.

In [6]:
colors = np.array(["FFFFFF","E4E4E4","888888","222222",
                   "FFA7D1","E50000","E59500","A06A42",
                   "E5D900","94E044","02BE01","00E5F0",
                   "0083C7","0000EA","E04AFF","820080"])

colordict = np.arange(3*16).reshape(16,3).astype(float)
for j in range(len(colors)):
    new = tuple(int(colors[j][i:i+2], 16)/255 for i in (0, 2, 4))
    colordict[j] = new
colordict

array([[1.        , 1.        , 1.        ],
       [0.89411765, 0.89411765, 0.89411765],
       [0.53333333, 0.53333333, 0.53333333],
       [0.13333333, 0.13333333, 0.13333333],
       [1.        , 0.65490196, 0.81960784],
       [0.89803922, 0.        , 0.        ],
       [0.89803922, 0.58431373, 0.        ],
       [0.62745098, 0.41568627, 0.25882353],
       [0.89803922, 0.85098039, 0.        ],
       [0.58039216, 0.87843137, 0.26666667],
       [0.00784314, 0.74509804, 0.00392157],
       [0.        , 0.89803922, 0.94117647],
       [0.        , 0.51372549, 0.78039216],
       [0.        , 0.        , 0.91764706],
       [0.87843137, 0.29019608, 1.        ],
       [0.50980392, 0.        , 0.50196078]])

## Task 2
We need to create some reusable functions for finding things such as:
1. A method for displaying a part of a canvas, or at least form the data in a way that can be later displayed.
2. All instances where the coords are in some defined shape.
3. An ordered list of all pixels placed by a certain user.
4. Filter out every instances where the pixel wasn't of an appropriate color. 

First we'll create a method that takes in some canvas coords and a timestamp and returns a numpy array that can be used to print that canvas part on a plot.

In [10]:
def canvas_region(x1,y1,x2,y2,time):
    width = x2-x1+1
    height = y2-y1+1
    matrix = np.zeros((height,width,3))
    trimmed_data = raw_data[(raw_data.x >= x1) & (raw_data.x <= x2) & (raw_data.y >= y1) & (raw_data.y <= y2) & (raw_data.ts <= time)].sort_values('ts',ascending=True)
    for index, row in trimmed_data.iterrows():
        matrix[row.y-y1,row.x-x1] = colordict[row.color]
    #for x in range(width):
    #    for y in range(height):
    #        position = trimmed_data[(trimmed_data.x == x+x1) & (trimmed_data.y == y+y1) & (trimmed_data.ts <= time)].sort_values('ts',ascending=False).head(1)
    #        matrix[y,x] = colordict[position.color] if (colordict[position.color].shape[0] != 0) else colordict[0]
    return matrix

In [ ]:
img = canvas_region(281,619,395,647,1491238724000)
imgplot = plt.imshow(img)